In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, Robot, bgr8_to_jpeg
import torchvision
import torch
import torchvision.transforms as transforms
import cv2
import PIL.Image
import os
import torch.nn.functional as F

In [2]:
camera = Camera.instance(width=224, height=224, capture_width=1080, capture_height=720, fps=30)

image_widget = widgets.Image(format='jpeg', width=224, height=224)

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [4]:
camera.unobserve_all()

In [ ]:
camera.stop()

### Forward model

In [3]:
model_forward = torchvision.models.resnet18(pretrained=False)
model_forward.avgpool = torch.nn.AdaptiveAvgPool2d((1, 1))
model_forward.fc = torch.nn.Linear(512, 1)
model_forward.load_state_dict(torch.load('best_steering_model_circuitlaunch_224.pth'))
device = torch.device('cuda')
model_forward = model_forward.to(device)
model_forward = model_forward.eval()

In [5]:
mean = torch.Tensor([0.485, 0.456, 0.406]).float().cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).float().cuda()

def preprocess(image):
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    #image = transforms.functional.to_grayscale(image, num_output_channels=3)
    #image = transforms.functional.resized_crop(image, crop_percent * 224, 0, 224 - crop_percent * 224, 224, (224, 224))
    image = transforms.functional.to_tensor(image).to(device).float()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [5]:
tensor = preprocess(camera.value)

In [7]:
import time

In [5]:
t0 = time.time()
for i in range(20):
    #tensor = preprocess(camera.value)
    output = model_forward(tensor)
t1 = time.time()

print((t1 - t0) / 20.0)

NameError: name 'time' is not defined

In [6]:
steering_slider = widgets.FloatSlider(min=-1.0, max=1.0, value=0.0, description='steering')

display(steering_slider)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [7]:
speed_slider = widgets.FloatSlider(min=0.0, max=0.4, step=0.001, value=0.0, description='speed')
steering_bias = widgets.FloatSlider(min=-0.3, max=0.3, step=0.0001, value=0.0, description='bias')

display(speed_slider)
display(steering_bias)

FloatSlider(value=0.0, description='speed', max=0.4, step=0.001)

FloatSlider(value=0.0, description='bias', max=0.3, min=-0.3, step=0.0001)

In [8]:
from IPython.display import display
import traitlets
import ipywidgets.widgets as widgets
import sys
sys.path.append('/opt/nvidia/jetson-gpio/lib/python/')
sys.path.append('/opt/nvidia/jetson-gpio/lib/python/Jetson/GPIO')
from adafruit_servokit import ServoKit

class Racecar(traitlets.HasTraits):
    
    throttle = traitlets.Float(default_value=0.0)
    steering = traitlets.Float(default_value=0.0)
    
    def __init__(self):
        self._kit = ServoKit(channels=16)
        self._steering_servo = self._kit.continuous_servo[0]
        self._throttle_motor = self._kit.continuous_servo[1]
        
    @traitlets.observe('throttle')
    def _on_throttle(self, change):
        self._throttle_motor.throttle = -change['new']
        
    @traitlets.validate('throttle')
    def _validate_throttle(self, proposal):
        if proposal['value'] > 1.0:
            return 1.0
        if proposal['value'] < -1.0:
            return -1.0
        return proposal['value']
        
    @traitlets.validate('steering')
    def _validate_steering(self, proposal):
        if proposal['value'] > 1.0:
            return 1.0
        if proposal['value'] < -1.0:
            return -1.0
        return proposal['value']
        
    @traitlets.observe('steering')
    def _on_steering(self, change):
        self._steering_servo.throttle = change['new']
        
car = Racecar()

In [9]:
kp = widgets.FloatSlider(min=0.0, max=2.0, step=0.001, value=0.4, description='kp')

display(kp)

FloatSlider(value=0.4, description='kp', max=2.0, step=0.001)

In [10]:
error = 0.0

def execute(change):
    global error, error_last, error_i
    image = change['new']
    image = preprocess(image)
    
    output = float(model_forward(image).detach().cpu()[0][0])
    steering = kp.value * output + steering_bias.value
    steering_slider.value = steering
    speed = speed_slider.value
    
    car.throttle = speed
    car.steering = steering
    
execute({'new': camera.value})

In [11]:
camera.observe(execute, names='value')

In [12]:
camera.unobserve(execute, names='value')

In [11]:
import time

camera.unobserve_all()
time.sleep(0.5)
robot.stop()

NameError: name 'robot' is not defined

In [ ]:
display(image_widget)
display(steering_slider)